In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5,6"

from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained("/storage1/fs1/yinjie.tang/Active/Shawn_Xiao/Qwen1.5-14B-Chat",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("/storage1/fs1/yinjie.tang/Active/Shawn_Xiao/Qwen1.5-14B-Chat")



Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
#pip install pandas feedparser openpyxl

import requests
import pandas as pd
from requests.utils import requote_uri

# Define your search parameters
keyword = 'large language model text mining'  # Update this with your desired search term
max_results = 200

# Construct the query parameters
params = {
    'search_query': f'all:{keyword}',
    'start': 0,
    'max_results': max_results
}

# The base URL for the arXiv API
base_url = 'http://export.arxiv.org/api/query'

# Make the request to the arXiv API with parameters that requests will encode
response = requests.get(base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.text

    # Use feedparser to parse the response
    import feedparser
    parsed_data = feedparser.parse(data)

    # Initialize a list to store each paper's information
    papers = []

    # Loop through each entry and extract information
    for entry in parsed_data.entries:
        title = entry.title
        authors = ', '.join(author.name for author in entry.authors)
        abstract = entry.summary

        # Append the information to the papers list
        papers.append({'Title': title, 'Authors': authors, 'Abstract': abstract})

    # Convert the list of papers to a DataFrame
    df = pd.DataFrame(papers)

    # Define the Excel file path
    excel_filename = f'{keyword}_arxiv_search_results.xlsx'

    # Save the DataFrame to an Excel file
    df.to_excel(excel_filename, index=False)

    print(f'Saved {len(papers)} papers to {excel_filename}')
else:
    print(f'Failed to retrieve data. HTTP Status Code: {response.status_code}')


Saved 200 papers to large language model text mining_arxiv_search_results.xlsx


In [ ]:
# Qwen reads abstract and identify knowledge

import pandas as pd
import os
import torch

# Assuming `tokenizer`, `model`, and `device` are already defined and initialized
# Example device initialization: device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to interact with the LLM API using the new method, now with customizable system prompts
def ask_questions(abstract, questions, system_prompts):
    responses = []
    for question, system_prompt in zip(questions, system_prompts):
        # Combine the question and abstract to form the prompt
        prompt_text = question + " " + str(abstract)
        
        # Prepare the messages for the new API, using a customizable system prompt
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt_text}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(device)
        
        # Generate response with the new API
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=5000
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        
        # Decode the generated response
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        responses.append(response)
    return responses

# Read the Excel file
file_path = excel_filename  # Replace with your file path
df = pd.read_excel(file_path)

# Define your questions and system prompts
questions = [
    " ",
    " "
]  # Update questions as needed

system_prompts = [
    "What is the main findings and conclusions in this study? Answer with 1 concise sentence.",
    "You are specialized for analyzing computer science paper abstracts, focusing on identifying entities relationships related to terminology, algorithm, data structure, model architecture. You should output the identified causal relationships between entities in combination pairs. The output strictly follows the format: (Entity A, Entity B), (Entity C, Entity D) ... with no additional text."
]  # Define system prompts corresponding to each question

# Process each abstract and store the responses
total_rows = len(df)
for i, row in df.iterrows():
    # Clear the console at the beginning of each iteration
    os.system('cls' if os.name == 'nt' else 'clear')

    responses = ask_questions(row['Abstract'], questions, system_prompts)
    df.at[i, 'Answer to Question 1'] = responses[0]
    df.at[i, 'Answer to Question 2'] = responses[1]

    # Show the responses
    print(f"Responses for Row {i+1}:")
    for j, response in enumerate(responses):
        print(f"Response to {questions[j]}: {response}")

    # Calculate and show the progress percentage
    progress = ((i + 1) / total_rows) * 100
    print(f"Progress: {progress:.2f}% completed")
    
# Save the updated DataFrame back to an Excel file
output_file_path = 'updated(Qwen1.5 14b)_'+keyword+'_causal.xlsx'  # Replace with your desired output file path
df.to_excel(output_file_path, index=False)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Responses for Row 1:
Response to  : The study suggests that very large language models (VLLMs) have the potential to become a unified and effective methodology for text mining, offering advantages over conventional methods, but also presenting challenges in their design and development.
Response to  : (text mining, language model), (VLLM, text mining methodology), (conventional methods, VLLM), (VLLM techniques, text mining).
Progress: 0.50% completed


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Responses for Row 2:
Response to  : The study finds that token-level legal argument mining using a Longformer model outperforms sentence-level classification in accuracy and offers greater flexibility for analyzing legal texts.
Response to  : (Longformer Model, Token-Level Classification), (Sentence-Level Text Classification, Token-Level Classification)
Progress: 1.00% completed


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Responses for Row 129:
Response to  : The study investigates the causes of AI hallucinations in large language models, examines their impact on various tasks, and proposes strategies to mitigate them in order to improve the reliability of LLMs and combat health-related fake news.
Response to  : (Problematic Phenomena, Hallucinations), (Text Generation Systems, LLMs),
(Question-Answering Systems, LLMs), (False or Misleading Information,
Propagation), (Paper, AI Hallucination Causes), (AI Hallucination Classification,
Tasks), (Machine Translation, Task), (Question and Answer, Task), (Dialog Systems,
Task), (Summarisation Systems, Task), (Knowledge Graph, LLMs), (Visual Question
Answer, Task), (Potential Strategies, Mitigating Hallucinations), (Overall
Reliability, LLMs), (HeReFaNMi Project, Health-Related Fake News Mitigation),
(NGI Search, Support), (Information Dissemination, Integrity), (Age of Evolving AI
Technologies, Safeguarding Integrity)
Progress: 64.50% completed


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


Responses for Row 130:
Response to  : The study proposes a novel framework to pre-train a generative language model using a knowledge graph-synthesized corpus to address the challenges of scarce biomedical entity distribution and lack of semantic connections in large language models, but does not find significant benefits from including specific types of knowledge (synonym, description, or relational) in the evaluation.
Response to  : (General Corpus, Biomedical Entities), (Biomedical Entities, Text Mining), (Biomedical Entities, Question Answering), (Large Language Models, Biomedical Entities), (Biomedical Entities, Training), (Biomedical Entities, Semantic Connection), (Biomedical Knowledge Graph, Large Language Models), (Relational Knowledge, Entities), (Catastrophic Forgetting, Former Methods), (Proposed Framework, Generative LLM), (Knowledge Graph, LLM Pre-Training), (Synonym Information, Inclusion), (Description Information, Inclusion), (Relational Information, Inclusion).
Progre

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


In [ ]:
# remove repeat words
import pandas as pd
import re

# Load an Excel file
df = pd.read_excel(output_file_path, engine='openpyxl')

# Fill NaN values in 'Response to New Question' column with zero
df['Answer to Question 2'] = df['Answer to Question 2'].fillna(0)

# Convert the column values to strings (to ensure compatibility with re.findall)
column_values = df['Answer to Question 2'].astype(str).tolist()

# Initialize an empty list to hold entities
entities = []

# Regular expression to match the pattern (entity A, entity B)
pattern = r'\(([^,]+), ([^\)]+)\)'

# Iterate over each cell in the column
for value in column_values:
    # Find all matches of the pattern in the cell
    matches = re.findall(pattern, value)
    # For each match, extend the entities list with the extracted entities
    for match in matches:
        entities.extend(match)  # This adds both entity A and entity B to the list

# Remove duplicates if necessary
entities = list(dict.fromkeys(entities))

# Join the entities with commas
entities_string = ', '.join(entities)

print(entities_string)

import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load your Excel file
file_path = output_file_path
df = pd.read_excel(file_path, engine='openpyxl')

# Assuming you have a list of entities and their embeddings already
entities = [entity.strip() for entity in entities_string.split(',')]
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(entities)

# Identify similar phrases and store them in a dictionary
similar_phrases = {}

# Calculate total iterations for progress tracking
total_iterations = sum(range(len(entities)))

# Initialize a counter to track progress
current_iteration = 0

for i in range(len(entities)):
    for j in range(i + 1, len(entities)):
        similarity = util.pytorch_cos_sim(embeddings[i], embeddings[j])
        if similarity.item() > 0.8:
            # Assuming entities[i] is the first phrase and entities[j] is the similar one
            similar_phrases[entities[j]] = entities[i]

        # Update the current iteration counter after each inner loop iteration
        current_iteration += 1

    # Print the percentage completed
    percentage_completed = (current_iteration / total_iterations) * 100
    print(f"Progress: {percentage_completed:.2f}%")

# Note: Printing progress in the inner loop might slow down your code execution,
# especially if 'entities' is very large. You might want to update the progress
# less frequently, for example, only after each completion of the outer loop.


# Specify the column you want to modify
specific_column = 'Answer to Question 2'

# Calculate total iterations for progress tracking (only for the specific column)
total_rows = len(df)
current_iteration = 0

# Iterate through the specific column to substitute similar phrases
for index, row in df.iterrows():
    current_iteration += 1
    # Print progress every 100 rows to avoid performance degradation
    if current_iteration % 100 == 0 or current_iteration == total_rows:
        progress_percentage = (current_iteration / total_rows) * 100
        print(f"Progress: {progress_percentage:.2f}% complete.")
    
    cell_value = str(row[specific_column])
    for similar, original in similar_phrases.items():
        # Check if the phrase contains 'Yarrowia', if so, skip substitution
        if 'Yarrowia' in cell_value or 'Yarrowia' in similar:
            continue
        if similar in cell_value:
            # Substitute similar phrase with the original phrase, ignoring errors if not found
            try:
                df.at[index, specific_column] = cell_value.replace(similar, original)
            except Exception as e:
                print(f"Error substituting phrase: {e}")
                continue

# Save the modified Excel file
modified_file_path = 'modified_' + file_path
df.to_excel(modified_file_path, index=False, engine='openpyxl')

print("Excel file has been modified and saved as:", modified_file_path)
